In [1]:
import camelot
import re

In [2]:
def pdf_tables(date):
    url = f"https://www.vs.saude.ms.gov.br/wp-content/uploads/{date['year']}/{date[ 'month']}/Boletim-Epidemiologico-COVID-19-{date['year']}.{date['month']}.{date['day']}.pdf"
    tables = camelot.read_pdf(url, pages="all", strip_text="▼, ▲, \n") # 18, 19, 20, 33, 34, 35, 36
    return tables

In [3]:
def add_new_row(data_frame, name_row, t_confirmado, t_obito):
    data_frame.loc[-1] = [name_row, t_confirmado, t_obito]  # adding a row
    data_frame.index = data_frame.index + 1  # shifting index
    return data_frame.sort_index()  # sorting by index

In [4]:
dates = ["2021-05-01"]
for date in dates:
    
    tables = pdf_tables({"year": date[:4], "month": date[5:7], "day": date[8:],})

    first_line = "MunicípioÓbitosDistribuiçãoLetalidadeMortalidade"
    for table in tables:
        try:
            if table.df.at[0, 2] == "Casosconfirmados":
                table.df = table.df.filter([0,2])
                table.df.drop([0], inplace=True)
                # Creat and appending table
                if not ('data_frame_confirmed' in vars() or 'data_frame_confirmed' in globals()):
                    data_frame_confirmed = table.df
                else:
                    data_frame_confirmed = data_frame_confirmed.append(table.df, ignore_index=True)


            elif table.df.at[0, 0] == first_line or table.df.at[0, 1] == first_line or table.df.at[0, 2] == first_line:
                table.df.drop(columns=[0, 4, 5, 6], inplace=True)

                table.df = table.df.drop([0]).reset_index()
                
                # Rows format
                for i in range(len(table.df[2])):
                    if table.df.at[i, 1] != "=" and table.df.at[i, 1] != "":
                        table.df.at[i, 2] = "".join(i for i in table.df.at[i, 1] if not i.isdigit())
                
                # Creat and appending table
                if not ('data_frame' in vars() or 'data_frame' in globals()):
                    data_frame = table.df
                else:
                    data_frame = data_frame.append(table.df, ignore_index=True)
                
                data_frame.drop(columns=[1, "index"], inplace=True)
        except:
            pass

    data_frame_confirmed = data_frame_confirmed.sort_values(by=0)
    data_frame_confirmed.reset_index(drop=True, inplace=True)

    confirmed_col = data_frame_confirmed[2]

    # Drop first row
    data_frame = data_frame.drop([0]).reset_index(drop=True)
    
    data_frame.rename(columns={2: "municipio", 3: "mortes"}, inplace=True)
    data_frame.sort_values(by="municipio", inplace=True)
    data_frame.reset_index(drop=True, inplace=True)

    # Inserting column at the
    # beginning in the DataFrame
    data_frame.insert(loc = 1,
        column = 'confirmados',
        value = confirmed_col)

    data_frame.reset_index(drop=True, inplace=True)
    
    # Adding space before every uppercase letter
    data_frame["municipio"] = data_frame["municipio"].apply(lambda x: re.sub(r"(\w)([A-Z])", r"\1 \2", x))
    # Adding space before every word + do + de + da
    data_frame["municipio"] = data_frame["municipio"].apply(lambda x: re.sub(r"(\w)((d)(\w)\s[A-Z])", r"\1 \2", x))
        
    # Remove accents
    data_frame["new"] = data_frame["municipio"].str.normalize('NFKD')\
             .str.encode('ascii', errors='ignore')\
             .str.decode('utf-8')
    
    # Sort by new column, then drop
    data_frame = data_frame.sort_values("new", ascending=True)\
        .drop("new", axis=1)
        
    # Generating csv
    data_frame.to_csv("test.csv", line_terminator=None, index=False)

,municipio,confirmados,mortes
0,Alcinópolis,167,6
1,Amambai,2.120,40
2,Anastácio,1.697,52
3,Anaurilândia,293,10
4,Angélica,631,14
...,...,...,...
74,Taquarussu,212,5
75,Terenos,955,38
76,Três Lagoas,13.800,315
77,Vicentina,555,14
